![](https://post.medicalnewstoday.com/wp-content/uploads/sites/3/2020/02/305190_2200-1200x628.jpg)


<p style='text-align: center;'><span style="color: #0D0D0D; font-family: Segoe UI; font-size: 2.6em; font-weight: 300;">[GPU] VINBIGDATA EFFICIENTDET TRAINING USING TF OBJECT DETECTION API </span></p>


<p style='text-align: justify;'><span style="color: #148998; font-family: Segoe UI; font-size: 2.1em; font-weight: 300;">Overview</span></p>

<p style='text-align: justify;'><span style="color: #19191d; font-family: Arial; font-size: 1.2em;">TensorFlow 2 Object Detection API is a framework built on top of TF2, which makes it easy to construct, train and deploy, robust and high performance Object Detection Models. This notebook presents the process of TFrecord Data Preparation, loading Pre-trained models, Transfer Learning, Tuning and Inference of state-of-the-art models such as EfficientDet.</span></p>

<p style='text-align: justify;'><span style="color: #19191d; font-family: Arial; font-size: 1.2em;">Serveral models from the TensorFlow Model Zoo can be trained with just 1-2 lines of code change.</span></p>

<br />

<img src="https://i.ibb.co/V9wc85F/tboard1.gif" width="813" height="357" />

<br />
<br />
<br />

<p style='text-align: justify;'><span style="color: #b02917; font-family: Segoe UI; font-size: 1.5em; font-weight: 300;">Please do check out the dataset used for training and the notebook for fusing bounding boxes with Weighted Boxes Fusion.</span></p>

<span style="font-family: Arial; font-size: 1.0em">VinBigData COCO Dataset 3x Downsampled</span>

DATASET LINK - https://www.kaggle.com/sreevishnudamodaran/vinbigdata-coco-dataset-with-wbf-3x-downscaled

<span style="font-family: Arial; font-size: 1.0em">VinBigData - Fusing Bboxes + Coco Dataset Creation Notebook</span>

NOTEBOOK LINK - https://www.kaggle.com/sreevishnudamodaran/vinbigdata-fusing-bboxes-coco-dataset

<br />
<br />

[![Ask Me Anything !](https://img.shields.io/badge/Ask%20me-anything-1abc9c.svg?style=flat-square&logo=kaggle)](https://www.kaggle.com/sreevishnudamodaran)

<br />

![Upvote!](https://img.shields.io/badge/Upvote-If%20you%20like%20my%20work-07b3c8?style=for-the-badge&logo=kaggle)

## Installations

In [ ]:
!pip install tf_slim

In [ ]:
## Make sure we have `pycocotools` installed
!pip install pycocotools
!pip install lvis
!pip install numba 

## Download and extract TensorFlow Model Garden or `cd` to it if it exists

Model Garden is an official TensorFlow repository on github.com which is a comprehensive repository of trained models ready for fine-tuning.

In [ ]:
import os
import pathlib

if "models" in pathlib.Path.cwd().parts:
    while "models" in pathlib.Path.cwd().parts:
        os.chdir('..')
elif not pathlib.Path('models').exists():
    !git clone --depth 1 https://github.com/tensorflow/models

## Compile Protobufs and Install the Object Detection Package


TensorFlow Object Detection API uses Protobuf library to configure the model and the training parameters.

In [ ]:
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.

In [ ]:
%%bash 
cd models/research
pip install .

## Set Environment Variabes

In [ ]:
os.environ['PYTHONPATH'] += ":/kaggle/working/models"

import sys
sys.path.append("/kaggle/working/models")

## Import Packages

In [ ]:
import numpy as np
import os
import sys
import tarfile
import tensorflow as tf
import zipfile
import cv2
import json
import pandas as pd
import glob
import os.path as osp
from path import Path
import datetime
import random
import shutil
from io import StringIO, BytesIO
from PIL import Image
from IPython.display import display
import re

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rcParams
sns.set(rc={"font.size":9,"axes.titlesize":15,"axes.labelsize":9,
            "axes.titlepad":11, "axes.labelpad":9, "legend.fontsize":7,
            "legend.title_fontsize":7, 'axes.grid' : False})

from sklearn.model_selection import train_test_split

## Import object detection module
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_utils
from object_detection.protos.string_int_label_map_pb2 import StringIntLabelMap, StringIntLabelMapItem
from object_detection.utils import config_util
from object_detection.builders import model_builder

from google.protobuf import text_format

import tarfile
from numba import cuda 

### Patches

In [ ]:
# patch tf1 into `utils.ops`
utils_ops.tf = tf.compat.v1

# Patch the location of gfile
tf.gfile = tf.io.gfile

# Load and Infer Using a Pre-trained Model

Any model exported using the `export_inference_graph.py` tool can be loaded here simply by changing the path.

See the [detection model zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md) for a list of other models that can be run out-of-the-box with varying speeds and accuracies.

## Loading Label Map
Label maps map indices to category names, so that when our convolution network predicts `5`, we know that this corresponds to `airplane`.  Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine.

In [ ]:
PATH_TO_LABELS = 'models/research/object_detection/data/mscoco_label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

## Download Model Files

**Replace this path with the path of your model of choice from the TF Model Garden**

In [ ]:
# !rm -r /kaggle/working/training_job/

In [ ]:
pretrained_dir = "/kaggle/working/training_job/model/"

if not os.path.exists(pretrained_dir):
    os.makedirs(pretrained_dir)
    print('Pretrainined Model Directory:', pretrained_dir)

In [ ]:
%%HTML
<style>
td {
  font-size: 13px
}
</style>

## Choosing Pre-trained Model & Downloading Weight Files

Choose your pre-trained model of choice. Please note that I have provided the paths of EfficientDet models in the cell below to choose any EfficientDet model. Apart from these, the TensorFlow Object Detection API provides numerous other models:
<br /> 

Visit the Model Zoo and simply replace the URL and the Model Name in the to get going.
https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md

<br /> 


| <div style="width:450px">Model Name</div>                               | Speed (ms) | COCO mAP | Outputs       |
| :---                                      |   :----:   |  :----:  |:----          |
|CenterNet HourGlass104 512x512	            |     70	 |   41.9	|Boxes          |
|CenterNet HourGlass104 Keypoints 512x512	|	76	|40.0/61.4|	Boxes/Keypoints|
|CenterNet HourGlass104 1024x1024|	197|	44.5	|Boxes|
|CenterNet HourGlass104 Keypoints 1024x1024|	211|	42.8/64.5	|Boxes/Keypoints|
|CenterNet Resnet50 V1 FPN 512x512|	27|	31.2	|Boxes|
|CenterNet Resnet50 V1 FPN Keypoints 512x512|	30|	29.3/50.7	|Boxes/Keypoints|
|CenterNet Resnet101 V1 FPN 512x512|	34|	34.2	|Boxes|
|CenterNet Resnet50 V2 512x512|	27|	29.5	|Boxes|
|CenterNet Resnet50 V2 Keypoints 512x512|	30|	27.6/48.2	|Boxes/Keypoints|
|EfficientDet D0 512x512|	39|	33.6	|Boxes|
|EfficientDet D1 640x640|	54|	38.4	|Boxes|
|EfficientDet D2 768x768|	67|	41.8	|Boxes|
|EfficientDet D3 896x896|	95|	45.4	|Boxes|
|EfficientDet D4 1024x1024|	133|	48.5	|Boxes|
|EfficientDet D5 1280x1280|	222|	49.7	|Boxes|
|EfficientDet D6 1280x1280|	268|	50.5	|Boxes|
|EfficientDet D7 1536x1536|	325|	51.2	|Boxes|
|SSD MobileNet v2 320x320|	19|	20.2 |Boxes|
|SSD MobileNet V1 FPN 640x640|	48|	29.1	|Boxes|
|SSD MobileNet V2 FPNLite 320x320|	22|	22.2	|Boxes|
|SSD MobileNet V2 FPNLite 640x640|	39|	28.2	|Boxes|
|SSD ResNet50 V1 FPN 640x640 (RetinaNet50)|	46|	34.3	|Boxes|
|SSD ResNet50 V1 FPN 1024x1024 (RetinaNet50)|	87|	38.3	|Boxes|
|SSD ResNet101 V1 FPN 640x640 (RetinaNet101)|	57|	35.6	|Boxes|
|SSD ResNet101 V1 FPN 1024x1024 (RetinaNet101)|	104|	39.5	|Boxes|
|SSD ResNet152 V1 FPN 640x640 (RetinaNet152)|	80|	35.4	|Boxes|
|SSD ResNet152 V1 FPN 1024x1024 (RetinaNet152)|	111|	39.6	|Boxes|
|Faster R-CNN ResNet50 V1 640x640|	53|	29.3	|Boxes|
|Faster R-CNN ResNet50 V1 1024x1024|	65|	31	|Boxes|
|Faster R-CNN ResNet50 V1 800x1333|	65|	31.6	|Boxes|
|Faster R-CNN ResNet101 V1 640x640|	55|	31.8	|Boxes|
|Faster R-CNN ResNet101 V1 1024x1024|	72|	37.1	|Boxes|
|Faster R-CNN ResNet101 V1 800x1333|	77|	36.6	|Boxes|
|Faster R-CNN ResNet152 V1 640x640|	64|	32.4	|Boxes|
|Faster R-CNN ResNet152 V1 1024x1024|	85|	37.6	|Boxes|
|Faster R-CNN ResNet152 V1 800x1333|	101|	37.4	|Boxes|
|Faster R-CNN Inception ResNet V2 640x640|	206|	37.7	|Boxes|
|Faster R-CNN Inception ResNet V2 1024x1024|	236|	38.7	|Boxes|
|Mask R-CNN Inception ResNet V2 1024x1024|	301|	39.0/34.6	|Boxes/Masks|
|ExtremeNet|--|	-- |Boxes|


In [ ]:
## EfficientDet Configurations

MODELS_CONFIG = {
    'efficientdet-d0': {
        'model_name': 'efficientdet_d0_coco17_tpu-32',
        'base_pipeline_file': 'ssd_efficientdet_d0_512x512_coco17_tpu-8.config',
        'pretrained_checkpoint': 'efficientdet_d0_coco17_tpu-32.tar.gz',
    },
    'efficientdet-d1': {
        'model_name': 'efficientdet_d1_coco17_tpu-32',
        'base_pipeline_file': 'ssd_efficientdet_d1_640x640_coco17_tpu-8.config',
        'pretrained_checkpoint': 'efficientdet_d1_coco17_tpu-32.tar.gz',
    },
    'efficientdet-d2': {
        'model_name': 'efficientdet_d2_coco17_tpu-32',
        'base_pipeline_file': 'ssd_efficientdet_d2_768x768_coco17_tpu-8.config',
        'pretrained_checkpoint': 'efficientdet_d2_coco17_tpu-32.tar.gz',
    },
    'efficientdet-d3': {
        'model_name': 'efficientdet_d3_coco17_tpu-32',
        'base_pipeline_file': 'ssd_efficientdet_d3_896x896_coco17_tpu-32.config',
        'pretrained_checkpoint': 'efficientdet_d3_coco17_tpu-32.tar.gz',
    },
    'efficientdet-d4': {
        'model_name': 'efficientdet_d4_coco17_tpu-32',
        'base_pipeline_file': 'ssd_efficientdet_d4_896x896_coco17_tpu-32.config',
        'pretrained_checkpoint': 'efficientdet_d4_coco17_tpu-32.tar.gz',
    },
    'efficientdet-d5': {
        'model_name': 'efficientdet_d5_coco17_tpu-32',
        'base_pipeline_file': 'ssd_efficientdet_d5_896x896_coco17_tpu-32.config',
        'pretrained_checkpoint': 'efficientdet_d5_coco17_tpu-32.tar.gz',
    },
    'efficientdet-d6': {
        'model_name': 'efficientdet_d6_coco17_tpu-32',
        'base_pipeline_file': 'ssd_efficientdet_d6_896x896_coco17_tpu-32.config',
        'pretrained_checkpoint': 'efficientdet_d6_coco17_tpu-32.tar.gz',
    },
    'efficientdet-d7': {
        'model_name': 'efficientdet_d7_coco17_tpu-32',
        'base_pipeline_file': 'ssd_efficientdet_d7_896x896_coco17_tpu-32.config',
        'pretrained_checkpoint': 'efficientdet_d7_coco17_tpu-32.tar.gz',
    }
}

## Choosing D1 here for this tutorial
chosen_model = 'efficientdet-d1'
model_name = MODELS_CONFIG[chosen_model]['model_name']
pretrained_checkpoint = MODELS_CONFIG[chosen_model]['pretrained_checkpoint']
base_pipeline_file = MODELS_CONFIG[chosen_model]['base_pipeline_file']

In [ ]:
model_url = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/'+pretrained_checkpoint
# pretrained_dir = "/kaggle/working/models/research/object_detection/pretrained"

!wget {model_url}
!tar -xf {pretrained_checkpoint}
!mv {model_name}/ {pretrained_dir}
!rm {pretrained_checkpoint}

model_dir = os.path.join(pretrained_dir, model_name, "saved_model")
print("Pre-trained model directory", model_dir)
model = tf.saved_model.load(str(model_dir))

## Display the Output Structure

In [ ]:
model.signatures['serving_default'].output_dtypes

In [ ]:
model.signatures['serving_default'].output_shapes

## Helper Functions

In [ ]:
def run_inference_for_single_image(model, image):
    '''
    Add a wrapper function to call the model, and cleanup the outputs:
    '''
    image = np.asarray(image)
    # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
    input_tensor = tf.convert_to_tensor(image)
    # The model expects a batch of images, so add an axis with `tf.newaxis`.
    input_tensor = input_tensor[tf.newaxis,...]

    # Run inference
    model_fn = model.signatures['serving_default']
    output_dict = model_fn(input_tensor)

    # All outputs are batches tensors.
    # Convert to numpy arrays, and take index [0] to remove the batch dimension.
    # We're only interested in the first num_detections.
    num_detections = int(output_dict.pop('num_detections'))
    output_dict = {key:value[0, :num_detections].numpy() for key,value in output_dict.items()}
    output_dict['num_detections'] = num_detections

    # detection_classes should be ints.
    output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)

    # Handle models with masks:
    if 'detection_masks' in output_dict:
        # Reframe the the bbox mask to the image size.
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
                  output_dict['detection_masks'], output_dict['detection_boxes'],
                   image.shape[0], image.shape[1])      
        detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5,
                                           tf.uint8)
        output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()

    return output_dict

def show_inference(model, image_path):
    # the array based representation of the image will be used later in order to prepare the
    # result image with boxes and labels on it.
    image_np = np.array(Image.open(image_path))
    # Actual detection.
    output_dict = run_inference_for_single_image(model, image_np)
    # Visualization of the results of a detection.
    vis_utils.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks_reframed', None),
      use_normalized_coordinates=True,
      line_thickness=8)

    return image_np
    
def load_image_into_numpy_array(path):
    """Load an image from file into a numpy array.

    Puts image into numpy array to feed into tensorflow graph.
    Note that by convention we put it into a numpy array with shape
    (height, width, channels), where channels=3 for RGB.

    Args:
    path: a file path.

    Returns:
    uint8 numpy array with shape (img_height, img_width, 3)
    """
    img_data = tf.io.gfile.GFile(path, 'rb').read()
    image = Image.open(BytesIO(img_data))
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

def plot_detections(image_np, boxes, classes, scores, category_index,
                    figsize=(12, 16), image_name=None):
    """Wrapper function to visualize detections.

    Args:
    image_np: uint8 numpy array with shape (img_height, img_width, 3)
    boxes: a numpy array of shape [N, 4]
    classes: a numpy array of shape [N]. Note that class indices are 1-based,
      and match the keys in the label map.
    scores: a numpy array of shape [N] or None.  If scores=None, then
      this function assumes that the boxes to be plotted are groundtruth
      boxes and plot all boxes as black with no classes or scores.
    category_index: a dict containing category dictionaries (each holding
      category index `id` and category name `name`) keyed by category indices.
    figsize: size for the figure.
    image_name: a name for the image file.
    """
    image_np_with_annotations = image_np.copy()
    viz_utils.visualize_boxes_and_labels_on_image_array(
      image_np_with_annotations,
      boxes,
      classes,
      scores,
      category_index,
      use_normalized_coordinates=True,
      min_score_thresh=0.8)
    if image_name:
        plt.imsave(image_name, image_np_with_annotations)
    else:
        plt.imshow(image_np_with_annotations)
        
        
def plot_img(img, size=(18, 18), is_rgb=True, title="", cmap='gray'):
    plt.figure(figsize=size)
    plt.imshow(img, cmap=cmap)
    plt.suptitle(title)

## Run the Model on Sample Images

In [ ]:
!wget https://cdn.pixabay.com/photo/2015/03/26/09/43/city-690158_960_720.jpg
!wget https://cdn.pixabay.com/photo/2017/08/05/23/31/people-2586656_960_720.jpg
!wget https://upload.wikimedia.org/wikipedia/commons/6/60/Naxos_Taverna.jpg
    
!wget https://cdn.pixabay.com/photo/2020/02/07/15/33/girl-4827500_960_720.jpg
TEST_IMAGE_PATHS = [
                    'girl-4827500_960_720.jpg',
                    'Naxos_Taverna.jpg',
                    'city-690158_960_720.jpg'
                   ]


In [ ]:
for image_path in TEST_IMAGE_PATHS:
    image = show_inference(model, image_path)
    os.remove(image_path)
    plot_img(image)
    plt.savefig('{}_viz.png'.format(image_path))
    plt.show()

### Delete Model and Release GPU Memory

In [ ]:
del model
device = cuda.get_current_device()
device.reset()

## TFRecord Creation

We are using the [vinbigdata-coco-dataset-with-wbf-3x-downscaled](https://www.kaggle.com/sreevishnudamodaran/vinbigdata-coco-dataset-with-wbf-3x-downscaled) dataset in COCO format with Fused Bounding Boxes using the Weighted Boxes Fusion Technique. This makes it easy to create TFRecords in just One Step.

Dataset Creation Notebook:

https://www.kaggle.com/sreevishnudamodaran/vinbigdata-fusing-bboxes-coco-dataset

### Create Ouput TFRecord Folder

In [ ]:
tfr_output_dir = "/kaggle/working/training_job/tfrecords/"

if not os.path.exists(tfr_output_dir):
    os.makedirs(tfr_output_dir)
    print('TFRecord Directory:', tfr_output_dir)

**The cell below uses create_coco_tf_record tool provided by TensorFlow converts COCO datasets into TFRecords directly.**

**Please note that the valications images and annotations are passed to the tool as the test data since the test argument is not optional. We will only be making use of Train and Validations TFRecords.**

In [ ]:
%%bash
python models/research/object_detection/dataset_tools/create_coco_tf_record.py --logtostderr \
--train_image_dir="../input/vinbigdata-coco-dataset-with-wbf-3x-downscaled/vinbigdata-coco-dataset-with-wbf-3x-downscaled" \
--test_image_dir="../input/vinbigdata-coco-dataset-with-wbf-3x-downscaled/vinbigdata-coco-dataset-with-wbf-3x-downscaled" \
--val_image_dir="../input/vinbigdata-coco-dataset-with-wbf-3x-downscaled/vinbigdata-coco-dataset-with-wbf-3x-downscaled" \
--train_annotations_file="../input/vinbigdata-coco-dataset-with-wbf-3x-downscaled/vinbigdata-coco-dataset-with-wbf-3x-downscaled/train_annotations.json" \
--testdev_annotations_file="../input/vinbigdata-coco-dataset-with-wbf-3x-downscaled/vinbigdata-coco-dataset-with-wbf-3x-downscaled/val_annotations.json" \
--val_annotations_file="../input/vinbigdata-coco-dataset-with-wbf-3x-downscaled/vinbigdata-coco-dataset-with-wbf-3x-downscaled/val_annotations.json" \
--output_dir="/kaggle/working/training_job/tfrecords/"

## Creating the Label Map File

This is just a file which contains the mapping of the classes and their integer ID. This is be used to internally to encode the train data and to decipher the model predictions during inference.

In [ ]:
def convert_classes(classes, start=1):
    msg = StringIntLabelMap()
    for id, name in enumerate(classes, start=start):
        msg.item.append(StringIntLabelMapItem(id=id, name=name))
    text = str(text_format.MessageToBytes(msg, as_utf8=True), 'utf-8')
    return text

labels =  [
            "Aortic_enlargement",
            "Atelectasis",
            "Calcification",
            "Cardiomegaly",
            "Consolidation",
            "ILD",
            "Infiltration",
            "Lung_Opacity",
            "Nodule_Mass",
            "Other_lesion",
            "Pleural_effusion",
            "Pleural_thickening",
            "Pneumothorax",
            "Pulmonary_fibrosis"
            ]

txt = convert_classes(labels)
print(txt)
with open('/kaggle/working/training_job/label_map.pbtxt', 'w') as f:
    f.write(txt)

# Preparing Train Job Folder 

In [ ]:
model_files_dir = "/kaggle/working/training_job/model_files/"

if not os.path.exists(model_files_dir):
    os.makedirs(model_files_dir)
    print('Model Files Directory:', model_files_dir)

In [ ]:
## Download pretrained weights
download_tar = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/' + pretrained_checkpoint
!wget {download_tar} -P {model_files_dir}
tar = tarfile.open(os.path.join(model_files_dir, pretrained_checkpoint))
tar.extractall()
tar.close()

In [ ]:
!rm {model_files_dir}/ssd_efficientdet_d2_768x768_coco17_tpu-8.config

In [ ]:
## Download base training configuration file

download_config = 'https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/configs/tf2/'+base_pipeline_file
!wget {download_config} -P {model_files_dir}

In [ ]:
## Get the Total Number of Classes

def get_num_classes(pbtxt_fname):
    from object_detection.utils import label_map_util
    label_map = label_map_util.load_labelmap(pbtxt_fname)
    categories = label_map_util.convert_label_map_to_categories(
        label_map, max_num_classes=90, use_display_name=True)
    category_index = label_map_util.create_category_index(categories)
    return len(category_index.keys())


val_record_fname = '/kaggle/working/training_job/tfrecords/coco_val.record*'
train_record_fname = '/kaggle/working/training_job/tfrecords/coco_train.record*'
label_map_pbtxt_fname = '/kaggle/working/training_job/label_map.pbtxt'

pipeline_file = os.path.join(model_files_dir, base_pipeline_file)
fine_tune_checkpoint = os.path.join(pretrained_dir, model_name, 'checkpoint/ckpt-0')
num_classes = get_num_classes(label_map_pbtxt_fname)
print("No. of Classes", num_classes)
print("Checkpoint File Path", fine_tune_checkpoint)

# Configuring the Model Training Pipeline File

## File Structure

This file represents the configuration and hyperparamters which will be used for the training job. It can be configured as required for further tuning.


```python
# SSD with EfficientNet-b1 + BiFPN feature extractor,
# shared box predictor and focal loss (a.k.a EfficientDet-d1).
# See EfficientDet, Tan et al, https://arxiv.org/abs/1911.09070
# See Lin et al, https://arxiv.org/abs/1708.02002
# Trained on COCO, initialized from an EfficientNet-b1 checkpoint.
#
# Train on TPU-8

model {
  ssd {
    inplace_batchnorm_update: true
    freeze_batchnorm: false
    num_classes: 90
    add_background_class: false
    box_coder {
      faster_rcnn_box_coder {
        y_scale: 10.0
        x_scale: 10.0
        height_scale: 5.0
        width_scale: 5.0
      }
    }
    matcher {
      argmax_matcher {
        matched_threshold: 0.5
        unmatched_threshold: 0.5
        ignore_thresholds: false
        negatives_lower_than_unmatched: true
        force_match_for_each_row: true
        use_matmul_gather: true
      }
    }
    similarity_calculator {
      iou_similarity {
      }
    }
    encode_background_as_zeros: true
    anchor_generator {
      multiscale_anchor_generator {
        min_level: 3
        max_level: 7
        anchor_scale: 4.0
        aspect_ratios: [1.0, 2.0, 0.5]
        scales_per_octave: 3
      }
    }
    image_resizer {
      keep_aspect_ratio_resizer {
        min_dimension: 640
        max_dimension: 640
        pad_to_max_dimension: true
        }
    }
    box_predictor {
      weight_shared_convolutional_box_predictor {
        depth: 88
        class_prediction_bias_init: -4.6
        conv_hyperparams {
          force_use_bias: true
          activation: SWISH
          regularizer {
            l2_regularizer {
              weight: 0.00004
            }
          }
          initializer {
            random_normal_initializer {
              stddev: 0.01
              mean: 0.0
            }
          }
          batch_norm {
            scale: true
            decay: 0.99
            epsilon: 0.001
          }
        }
        num_layers_before_predictor: 3
        kernel_size: 3
        use_depthwise: true
      }
    }
    feature_extractor {
      type: 'ssd_efficientnet-b1_bifpn_keras'
      bifpn {
        min_level: 3
        max_level: 7
        num_iterations: 4
        num_filters: 88
      }
      conv_hyperparams {
        force_use_bias: true
        activation: SWISH
        regularizer {
          l2_regularizer {
            weight: 0.00004
          }
        }
        initializer {
          truncated_normal_initializer {
            stddev: 0.03
            mean: 0.0
          }
        }
        batch_norm {
          scale: true,
          decay: 0.99,
          epsilon: 0.001,
        }
      }
    }
    loss {
      classification_loss {
        weighted_sigmoid_focal {
          alpha: 0.25
          gamma: 1.5
        }
      }
      localization_loss {
        weighted_smooth_l1 {
        }
      }
      classification_weight: 1.0
      localization_weight: 1.0
    }
    normalize_loss_by_num_matches: true
    normalize_loc_loss_by_codesize: true
    post_processing {
      batch_non_max_suppression {
        score_threshold: 1e-8
        iou_threshold: 0.5
        max_detections_per_class: 100
        max_total_detections: 100
      }
      score_converter: SIGMOID
    }
  }
}

train_config: {
  fine_tune_checkpoint: "PATH_TO_BE_CONFIGURED/ckpt-0"
  fine_tune_checkpoint_version: V2
  fine_tune_checkpoint_type: "classification"
  batch_size: 128
  sync_replicas: true
  startup_delay_steps: 0
  replicas_to_aggregate: 8
  use_bfloat16: true
  num_steps: 300000
  data_augmentation_options {
    random_horizontal_flip {
    }
  }
  data_augmentation_options {
    random_scale_crop_and_pad_to_square {
      output_size: 640
      scale_min: 0.1
      scale_max: 2.0
    }
  }
  optimizer {
    momentum_optimizer: {
      learning_rate: {
        cosine_decay_learning_rate {
          learning_rate_base: 8e-2
          total_steps: 300000
          warmup_learning_rate: .001
          warmup_steps: 2500
        }
      }
      momentum_optimizer_value: 0.9
    }
    use_moving_average: false
  }
  max_number_of_boxes: 100
  unpad_groundtruth_tensors: false
}

train_input_reader: {
  label_map_path: "PATH_TO_BE_CONFIGURED/label_map.txt"
  tf_record_input_reader {
    input_path: "PATH_TO_BE_CONFIGURED/train2017-?????-of-00256.tfrecord"
  }
}

eval_config: {
  metrics_set: "coco_detection_metrics"
  use_moving_averages: false
  batch_size: 1;
}

eval_input_reader: {
  label_map_path: "PATH_TO_BE_CONFIGURED/label_map.txt"
  shuffle: false
  num_epochs: 1
  tf_record_input_reader {
    input_path: "PATH_TO_BE_CONFIGURED/val2017-?????-of-00032.tfrecord"
  }
}
    
```

## Define Parameters

In [ ]:
## Train Images = 3296
## 3296/8 = 412
## Val Images = 1098
## 1098/8 ~= 138

## Training Configurations
num_epochs = 5
num_steps = 412*num_epochs
num_eval_steps = 138
batch_size = 8
print("Number of Steps:", num_steps)

## Modify the Pipeline Configuration File

In [ ]:
#write custom configuration file by slotting our dataset, model checkpoint, and training parameters into the base pipeline file

print('Updading the Training Configuration file')
with open(pipeline_file) as f:
    s = f.read()
    
with open(pipeline_file, 'w') as f:
    
    # fine_tune_checkpoint
    s = re.sub('fine_tune_checkpoint: ".*?"',
               'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), s)
    
    # tfrecord files train and test.
    s = re.sub('(input_path: ".*?)(PATH_TO_BE_CONFIGURED/train)(.*?")',
               'input_path: "{}"'.format(train_record_fname), s)
    
    s = re.sub('(input_path: ".*?)(PATH_TO_BE_CONFIGURED/val)(.*?")',
               'input_path: "{}"'.format(val_record_fname), s)

    # label_map_path
    s = re.sub('label_map_path: ".*?"',
               'label_map_path: "{}"'.format(label_map_pbtxt_fname), s)

    # Set training batch_size.
    s = re.sub('batch_size: [0-9]+',
               'batch_size: {}'.format(batch_size), s)

    # Set training steps, num_steps
    s = re.sub('num_steps: [0-9]+',
               'num_steps: {}'.format(num_steps), s)
    
    # Set number of classes num_classes.
    s = re.sub('num_classes: [0-9]+',
               'num_classes: {}'.format(num_classes), s)
    
    
    #fine-tune checkpoint type
    s = re.sub('fine_tune_checkpoint_type: "classification"',
               'fine_tune_checkpoint_type: "{}"'.format('detection'), s)
        
    f.write(s)

In [ ]:
## Check the pipeline config file
!cat {pipeline_file}

# Train Custom TF2 Object Detector




In [ ]:
model_dir = "/kaggle/working/training_job/model_files/training/"

if not os.path.exists(model_dir):
    os.makedirs(model_dir)
    print('Training Files:', model_dir)

In [ ]:
## Checking GPU Load
!nvidia-smi

## Start Training


model_dir: the location tensorboard logs and saved model checkpoints will save to

In [ ]:
!python /kaggle/working/models/research/object_detection/model_main_tf2.py \
    --pipeline_config_path={pipeline_file} \
    --model_dir={model_dir} \
    --alsologtostderr \
    --num_train_steps={num_steps} \
    --sample_1_of_n_eval_examples=1 \
    --num_eval_steps={num_eval_steps}

# Model Evaluation & Sample Inference
## Perform Evaluation using Model Main

In [ ]:
import glob
import os
os.environ['PYTHONPATH'] += ":/kaggle/working/models"
import sys
sys.path.append("/kaggle/working/models")

In [ ]:
model_dir = "/kaggle/working/training_job/model_files/training/"
pipeline_file = glob.glob(os.path.join('/kaggle/working/training_job/model_files/', '*.config'))[0]
pipeline_file

In [ ]:
## Stop this cell manually
!python /kaggle/working/models/research/object_detection/model_main_tf2.py \
     --pipeline_config_path={pipeline_file} \
     --model_dir={model_dir} \
     --checkpoint_dir={model_dir} \
     --run_once=True

## Reloading Packages & Env Variables

Running the model training and evaluation scripts will unload all the packages loaded. Please note that it does not reset the environment. No new installations need to be done.

In [ ]:
import os
os.environ['PYTHONPATH'] += ":/kaggle/working/models"

import sys
sys.path.append("/kaggle/working/models")

In [ ]:
import numpy as np

import sys
import cv2
import glob
import os.path as osp
from path import Path
import random
import io
import scipy.misc
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rcParams
sns.set(rc={"font.size":9,"axes.titlesize":15,"axes.labelsize":9,
            "axes.titlepad":11, "axes.labelpad":9, "legend.fontsize":7,
            "legend.title_fontsize":7, 'axes.grid' : False})

import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder


In [ ]:
model_dir = "/kaggle/working/training_job/model_files/training/"
pipeline_file = glob.glob(os.path.join('/kaggle/working/training_job/model_files/', '*.config'))[0]
pipeline_file

In [ ]:
## See our saved weights
!ls {model_dir}

## Monitoring & Evaluation
### Unfortunately running Tensorboard on Kaggle Notebooks directly is put on hold at the moment but, there is way to see it using ngrox!

**Thanks to [shivam1600](https://www.kaggle.com/shivam1600)**
<br />
https://www.kaggle.com/shivam1600/tensorboard-on-kaggle

### Uncomment the cells below to get tensorboard. It only works in editing mode and in a new tab.

In [ ]:
# # Download Ngrok to tunnel the tensorboard port to an external port
# !wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
# !unzip ngrok-stable-linux-amd64.zip

In [ ]:
# import multiprocessing

# log_path = os.path.join(model_dir, 'train')
# print("log_path", log_path)

# # Run tensorboard as well as Ngrox (for tunneling as non-blocking processes)
# pool = multiprocessing.Pool(processes = 10)
# results_of_processes = [pool.apply_async(os.system, args=(cmd, ), callback = None )
#                         for cmd in [
#                         f"tensorboard --logdir {log_path} --host 0.0.0.0 --port 6006 &",
#                         "./ngrok http 6006 &"
#                         ]]

### Click the link in the output of the cell below after running the notebook to view Tensorboard

In [ ]:
# !curl -s http://localhost:4040/api/tunnels | python3 -c \
#     "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

In [ ]:
# !ps -ef | grep 6006

<img src="https://i.ibb.co/V9wc85F/tboard1.gif" width="813" height="357" />

## Load the Latest Checkpoint

In [ ]:
last_ckpt = Path(sorted(glob.glob(os.path.join(model_dir,'ckpt-*')))[-1]).stem
last_ckpt

In [ ]:
configs = config_util.get_configs_from_pipeline_file(pipeline_file)
model_config = configs['model']
detection_model = model_builder.build(model_config=model_config, is_training=False)

## Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(model_dir, last_ckpt))

def get_model_detection_function(model):
    """Get a tf.function for detection."""
    @tf.function
    def detect_fn(image):
        """Detect objects in image."""
        image, shapes = model.preprocess(image)
        prediction_dict = model.predict(image, shapes)
        detections = model.postprocess(prediction_dict, shapes)
        return detections, prediction_dict, tf.reshape(shapes, [-1])
    return detect_fn

detect_fn = get_model_detection_function(detection_model)

## Load Label Map File For Scoring & Visualization

In [ ]:
## Map labels for Inference
label_map_path = configs['eval_input_config'].label_map_path
label_map = label_map_util.load_labelmap(label_map_path)
categories = label_map_util.convert_label_map_to_categories(
                label_map,
                max_num_classes=label_map_util.get_max_label_map_index(label_map),
                use_display_name=True)
category_index = label_map_util.create_category_index(categories)
label_map_dict = label_map_util.get_label_map_dict(label_map, use_display_name=True)

## Score on Sample Images

### Helper Functions for Visualization

In [ ]:
def plot_imgs(imgs, cols=2, size=10, is_rgb=True, title="", cmap='gray', img_size=None):
    rows = len(imgs)//cols + 1
    fig = plt.figure(figsize=(round(cols*size*0.7), rows*size))
    for i, img in enumerate(imgs):
        if img_size is not None:
            img = cv2.resize(img, img_size)
        fig.add_subplot(rows, cols, i+1)
        plt.imshow(img, cmap=cmap)
    plt.suptitle(title)   
    
def draw_bbox(image, box, label, color):   
    alpha = 0.1
    alpha_box = 0.4
    overlay_bbox = image.copy()
    overlay_text = image.copy()
    output = image.copy()
    output = cv2.rectangle(output, (box[0], box[1]), (box[2], box[3]),
                           color, 2)

    text_width, text_height = cv2.getTextSize(label.upper(), cv2.FONT_HERSHEY_SIMPLEX, 0.6, 1)[0]
    cv2.rectangle(overlay_bbox, (box[0], box[1]), (box[2], box[3]),
                  color, -1)
    cv2.addWeighted(overlay_bbox, alpha, output, 1 - alpha, 0, output)
    
    cv2.rectangle(overlay_text, (box[0], box[1]-7-text_height),
                  (box[0]+text_width+2, box[1]), (0, 0, 0), -1)
    
    cv2.addWeighted(overlay_text, alpha_box, output, 1 - alpha_box, 0, output)
    cv2.putText(output, label.upper(), (box[0], box[1]-5),
            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2, cv2.LINE_AA)
    return output

In [ ]:
val_images = glob.glob('../input/vinbigdata-coco-dataset-with-wbf-3x-downscaled/vinbigdata-coco-dataset-with-wbf-3x-downscaled/val_images/*.jpg')

viz_images = []
for img_path in val_images[:4]:
    image_np = cv2.imread(img_path)
    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0),
                                        dtype=tf.float32)
    detections, predictions_dict, shapes = detect_fn(input_tensor)
    label_id_offset = 1
    image_np_with_detections = image_np.copy()

    label2color = [[59, 238, 119], [222, 21, 229], [94, 49, 164], [206, 221, 133], [117, 75, 3],
                 [210, 224, 119], [211, 176, 166], [63, 7, 197], [102, 65, 77], [194, 134, 175],
                 [209, 219, 50], [255, 44, 47], [89, 125, 149], [110, 27, 100]]
    
    min_score_thresh = 0.4
    width, height, _ = image_np.shape
    boxes = detections['detection_boxes'][0].numpy()
    boxes = boxes[:,[1, 0, 3, 2]]*np.array([height, width, height, width])
    
    for box, label_id, score in zip(boxes,
                                    (detections['detection_classes'][0].numpy() + label_id_offset).astype(int),
                                     detections['detection_scores'][0].numpy()):
        if (score > min_score_thresh):
            image_np_with_detections = draw_bbox(image_np_with_detections, list(np.int_(box)),
                                                 category_index[label_id]['name']+'_'+str(round(score*100,1))+'%',
                                                 label2color[label_id-1])
        
    viz_images.append(image_np_with_detections)

plot_imgs(viz_images, cols=2, size=10, cmap=None)
plt.savefig('inference_sample.png')
plt.show()

In [ ]:
# !rm -r ./models
# !rm -r ./efficientdet_d1_coco17_tpu-32

<p style='text-align: center;'><span style="color: #0D0D0D; font-family: Segoe UI; font-size: 2.0em; font-weight: 200;">THANK YOU! PLEASE UPVOTE</span></p>

<p style='text-align: center;'><span style="color: #0D0D0D; font-family: Segoe UI; font-size: 2.5em; font-weight: 100;">HOPE IT WAS USEFUL</span></p>

<p style='text-align: center;'><span style="color: #009BAE; font-family: Segoe UI; font-size: 1.3em; font-weight: 300;">Tuning & updation in progress.</span></p>